In [3]:
%matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
# import gmplot
# from geopy.geocoders import Nominatim
# import time
# import gmaps
# import gmaps.datasets

Aijobsindustry_dataset = pd.read_csv("AIJobsIndustry_Clean_v3.csv")
print(f"No of rows in Aijobsindustry_dataset is {Aijobsindustry_dataset.shape[0]}")

Dice_us_jobs_dataset = pd.read_csv("Dice_US_jobs_Clean_v2.csv", encoding="cp437")
print(f"No of rows in Dice_us_jobs_dataset is {Dice_us_jobs_dataset.shape[0]}")

monster_dataset = pd.read_csv("Monster_Clean.csv")
print(f"No of rows in monster_dataset is {monster_dataset.shape[0]}")

indeed_dataset = pd.read_csv("alldata_Clean_v2.csv")
print(f"No of rows in indeed_dataset is {indeed_dataset.shape[0]}")

##file needs to match format for concat
# Dice_use_Tech_filtered = pd.read_csv("Dice_US_Tech_FilteredJobData.csv")
# Dice_use_Tech_filtered

print(f"No of rows in final merged data set should be is {Aijobsindustry_dataset.shape[0]+Dice_us_jobs_dataset.shape[0]+monster_dataset.shape[0]+indeed_dataset.shape[0]}")

datasets_to_merge = [Aijobsindustry_dataset,Dice_us_jobs_dataset,monster_dataset,indeed_dataset]

##please confirm the concat makes sense
merged_dataset = pd.concat(datasets_to_merge)
print(f"Actual no of rows in merged_dataset is {merged_dataset.shape[0]}")
merged_dataset


Using matplotlib backend: Qt5Agg
No of rows in Aijobsindustry_dataset is 631


FileNotFoundError: File b'Dice_US_jobs_Clean_v2.csv' does not exist

In [10]:
def parser(column):
    word_count = {}
    for entry in column:
        entry = (re.sub('[^A-Za-z0-9 ]+', '', str(entry))).lower()
        try:
            entry = entry.replace(',','')
            word_split = entry.split()
            for word in word_split:
                if word not in word_count:
                    word_count[word] = 1
                else:
                    word_count[word] += 1
        except AttributeError:
            print("Skipping an entry due to bad data")
            continue
    return word_count

In [7]:
desc = merged_dataset['Description']
parsed_data = parser(desc)
print(parsed_data)

{'nan': 631, 'informatica': 323, 'etl': 1324, 'developerst': 1, 'petersburg': 5, 'fl': 35, 'only': 1031, 'full': 1200, 'time': 3149, 'direct': 1110, 'hiresmultiple': 1, 'positionsrequirements5': 1, 'years': 10806, 'experience': 24043, 'as': 24472, 'a': 69060, 'di': 11, 'developerinformatica': 1, 'power': 1087, 'center': 1748, 'or': 36885, 'similar': 898, 'etldi': 1, 'toolinformatica': 1, 'b2b': 113, 'data': 44628, 'exchange': 146, 'soa': 46, 'toolunixlinux': 1, 'operating': 760, 'system': 3523, 'and': 240322, 'scripting': 776, 'languagedatabases': 1, 'netezza': 25, 'oracle': 660, 'mysql': 451, 'sqldata': 4, 'warehousing': 338, 'experiencestrong': 165, 'analytical': 3888, 'skills': 10304, 'presentation': 1015, 'skilldesiredexperience': 1, 'with': 62511, 'database': 3103, 'administration': 821, 'working': 7912, 'frontend': 274, 'tools': 5046, 'reporting': 2258, 'etcc': 1, 'java': 1635, 'c': 1392, 'programming': 2918, 'languagegood': 1, 'understanding': 4434, 'of': 101509, 'application': 

In [8]:
print(parsed_data['python'])

3610


In [ ]:
group_by_state = monster_dataset.groupby("Loc_state")
count_for_state = group_by_state.count()
count_for_state = count_for_state["Position"]
count_for_state